In [1]:
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.odr import RealData, ODR, Model as ODRModel
import logging

# Set up basic logging to see the output
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def read_json_data(json_path):
    """
    Reads scintillation data from a pipeline-generated JSON file.

    Args:
        json_path (str): The path to the input JSON file.

    Returns:
        tuple: A tuple containing arrays for frequencies (MHz), bandwidths (MHz),
               bandwidth errors (MHz), modulation indices, and modulation index errors.
               Returns (None, None, None, None, None) if an error occurs.
    """
    logging.info(f"Reading data from JSON file: {json_path}")
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)

        # Navigate to the sub-band measurements. We assume the main component is 'scint_scale'.
        # This might need to be adjusted if the component name changes.
        measurements = data['components']['scint_scale']['subband_measurements']

        freqs = np.array([m['freq_mhz'] for m in measurements])
        bws = np.array([m['bw'] for m in measurements])
        mods = np.array([m['mod'] for m in measurements])

        # Combine statistical fit error and finite scintle error in quadrature
        bw_err_fit = np.nan_to_num(np.array([m.get('bw_err') for m in measurements]))
        bw_err_finite = np.nan_to_num(np.array([m.get('finite_err') for m in measurements]))
        bw_errs = np.sqrt(bw_err_fit**2 + bw_err_finite**2)
        
        mod_errs = np.nan_to_num(np.array([m.get('mod_err') for m in measurements]))

        return freqs, bws, bw_errs, mods, mod_errs

    except (FileNotFoundError, KeyError, TypeError) as e:
        logging.error(f"Could not read or parse JSON file {json_path}. Error: {e}")
        return None, None, None, None, None

def read_pkl_data(pkl_path, model_key='1_lorenz'):
    """
    Reads scintillation data from a pickle file containing analysis results.

    Args:
        pkl_path (str): The path to the input pickle file.
        model_key (str): The key for the desired model fit ('1_lorenz' or '2_lorenz').
                         Defaults to '1_lorenz' for a single scintillation scale.

    Returns:
        tuple: A tuple containing arrays for frequencies (MHz), bandwidths (MHz),
               bandwidth errors (MHz), modulation indices, and modulation index errors.
               Returns (None, None, None, None, None) if an error occurs.
    """
    logging.info(f"Reading data from pickle file: {pkl_path}")
    try:
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        print(data)

        # Extract data based on the chosen model key
        if model_key not in data:
            logging.error(f"Model key '{model_key}' not found in pickle file.")
            return None, None, None, None, None
            
        freqs = np.array(data['f_cents'])
        
        # For a 1-component model, the keys are straightforward
        if model_key == '1_lorenz':
            bws = np.array(data[model_key]['sub_scint_1'])
            mods = np.array(data[model_key]['mods1'])
            bw_err_fit = np.nan_to_num(np.array(data[model_key]['sub_scint_uncert_1']))
            mod_errs = np.nan_to_num(np.array(data[model_key]['mods1_uncert']))
            
            # The 'add_un1' seems to be an additional uncertainty. We'll take its mean value
            # for each sub-band and add it in quadrature to the fit error.
            bw_err_add = np.array([np.mean(err) for err in data[model_key]['add_un1']])
            bw_errs = np.sqrt(bw_err_fit**2 + bw_err_add**2)

        # For a 2-component model, we must choose which component to use.
        # Here, we default to the broader component (sub_scint_2), a common choice.
        elif model_key == '2_lorenz':
            bws = np.array(data[model_key]['sub_scint_2']) # Choosing the second component
            mods = np.array(data[model_key]['mods2'])
            bw_err_fit = np.nan_to_num(np.array(data[model_key]['sub_scint_uncert_2']))
            mod_errs = np.nan_to_num(np.array(data[model_key]['mods2_uncert']))
            bw_err_add = np.array([np.mean(err) for err in data[model_key]['add_un2']])
            bw_errs = np.sqrt(bw_err_fit**2 + bw_err_add**2)
        
        else:
             raise ValueError(f"Unsupported model key: {model_key}")


        return freqs, bws, bw_errs, mods, mod_errs

    except (FileNotFoundError, KeyError, TypeError, pickle.UnpicklingError) as e:
        logging.error(f"Could not read or parse pickle file {pkl_path}. Error: {e}")
        return None, None, None, None, None




In [4]:
out = read_pkl_data('wilhelm_253635173_subband_acf_fits.pkl')

INFO: Reading data from pickle file: wilhelm_253635173_subband_acf_fits.pkl


{'1_lorenz': {'sub_scint_1': [0.239177755414001, 0.03098038269728807, 0.08902349045914636, 0.03060903481854996], 'sub_scint_uncert_1': [0.16924869256962413, 0.014594476244811576, 0.06139504876062839, 0.01827234228783935], 'mods1': [0.3005029817572613, 0.23880882627485556, 0.21708979235951498, 0.2878815369055432], 'mods1_uncert': [0.09866257813328634, 0.03351959254352127, 0.03441208093848267, 0.045683834554485186], 'add_un1': [array([0.02495628, 0.02794008, 0.03255887, 0.03098943]), array([0.0011861 , 0.00133459, 0.00156928, 0.00148882]), array([0.00574612, 0.00645617, 0.00757168, 0.00719022]), array([0.00116488, 0.00131072, 0.00154125, 0.00146221])], 'c1': [-0.065685558535165, -0.004152302144561304, -0.006184531100899449, 0.09701273817215395], 'c1_uncert': [0.06281885388372695, 0.004767737658598892, 0.013492130526070114, 0.010115450305916379]}, '2_lorenz': {'sub_scint_1': [0.00307364980666266, 0.025709408346127994, 0.08895984789247124, 0.030597509878495525], 'sub_scint_uncert_1': [0.02

In [5]:
print(out)

(array([727.9774772 , 693.3742263 , 674.49903223, 639.57916025]), array([0.23917776, 0.03098038, 0.08902349, 0.03060903]), array([0.17173404, 0.01466097, 0.06176402, 0.01832361]), array([0.30050298, 0.23880883, 0.21708979, 0.28788154]), array([0.09866258, 0.03351959, 0.03441208, 0.04568383]))


In [21]:
# In an environment that has lmfit & uncertainties installed
import pickle, lmfit, uncertainties
with open("wilhelm_253635173_subband_acf_fits.pkl", "rb") as f:
    acf_dict = pickle.load(f)

#print(acf_dict.keys())  # likely: dict_keys(['sub_scint_1', 'sub_scint_2', ...])
#print(acf_dict['2_lorenz'])
print(acf_dict['f_cents'])
print(acf_dict['1_lorenz']['sub_scint_1'])    # lmfit.ModelResult or similar
print(acf_dict['1_lorenz']['sub_scint_uncert_1'])
#print(acf_dict['2_lorenz']['sub_scint_2'])

[727.977477196368, 693.3742262989571, 674.4990322254934, 639.5791602472219]
[0.239177755414001, 0.03098038269728807, 0.08902349045914636, 0.03060903481854996]
[0.16924869256962413, 0.014594476244811576, 0.06139504876062839, 0.01827234228783935]
